# Droplet spreading over a flat surface against static contact angle $\theta_e$ -         Mesh convergence study
![Droplet_Spreading domain configuration](DropletSpreading.png)

For this case study, we have considered a droplet spreading on a flat surface ([dupont2010numerical](https://doi.org/10.1016/j.jcp.2009.07.034)) that allows for studying the effect of the static contact angle boundary condition and the Bond number, $Bo = \frac{\rho_l g R_0}{\sigma}$ on the equilibrium shape of the droplet. Ideally, for a droplet that spreads with $Bo\ll1$, surface tension forces dominate, and the droplet at equilibrium maintains a spherical cap shape and satisfies the contact angle boundary condition. On the other hand, for $Bo\gg1$, the gravitational forces dominate, and the droplet forms a puddle, whose height is directly proportional to the capillary length, $l_{Ca}=\sqrt{\frac{\sigma}{\rho_l g}}$. The conservation of the droplet’s volume $V$ with an equilibrium contact angle $\theta_e$ allows formulating the geometrical relations that define the equilibrium shape of the droplet ([dupont2010numerical](https://doi.org/10.1016/j.jcp.2009.07.034), [fricke2020geometry](https://arxiv.org/abs/2003.04914)). In addition to observing droplets at equilibrium, we have also studied the mesh convergence of the spreading droplets.

This notebook shows the mesh convergence of water_glycerol and water droplets against $\theta_e$.
#### Note: water_glycerol and water are labelled as 'wg' and 'w' respectively.

In [ ]:
#Wetted Area 
#Loading the necessary files
import pandas as pd
import numpy as np
from vtk import vtkStructuredPointsReader
from vtk.util import numpy_support as VN
import csv
import sys
# for radians
import math
import os
import matplotlib.pyplot as plt
import re
import itertools
from matplotlib import rcParams
pi = np.pi
sin = np.sin
cos = np.cos

rcParams["figure.dpi"] = 600
fontSize = 8


sys.path.append(os.path.join(os.getcwd(), '../../modules'))
print('modules path: ',os.path.join(os.getcwd(), '../../modules'))
from flat_spreading_funcs import Funcs

surfaceTensionModel = 'RDF' #surface tension model chosen for the simulations {heightFunction, RDF, fitParaboloid}
dataFolder = '/postProcessing/' #folder with .csv files


### Mesh convergence study for water_glycerol with contact angle $\theta_e=70^\circ$

In [ ]:
fluid = 'water_glycerol'
fluidLabel = 'wg'
contactAngle = 70
theta = np.radians(contactAngle)
pattern = fluidLabel+'test'+str(contactAngle)+'_'
dropletRadius = 0.001 #1mm
domainLength = 0.005


nX= [] # static contact angles used for simulations taken from the variation file
stm = [] # surface tension models in all cases,taken from the variation file
var_list = [] # variance number of the parametric study of a specific case

#read the variation-file to get the parameters map that is saved using pyFOAM
var_Map = Funcs.fileMapping(pattern, surfaceTensionModel)
print('variations map: ', var_Map)    
     
for key, value in var_Map.items():
    stm_map = str(value).split(', ')[0][:-1] #surface tension model map
    stm.append(stm_map.split(': ')[1][1:])
    nX_map = str(value).split(', ')[3][:-1] #cells in x-direction map
    nX.append(int(nX_map.split(': ')[1][0:]))
    var_list.append(str(key))  
print('number of cells in x-direction: ', nX)
print('Variant\'s list: ', var_list)

cellsPerRadius = [int(nx/domainLength*dropletRadius) for nx in nX]
print('cells per radius: ',cellsPerRadius)


#######################################################################################

wettedAreaFileNames = Funcs.fileStructure(dataFolder, 'wettedArea.csv', pattern, var_list )
print (wettedAreaFileNames)
plotLabels = ['cells per radius = ' + str(x) for x in cellsPerRadius]

#Reference solution 
plt.axhline(Funcs.getAnalyticalWettedArea(theta,dropletRadius), color='black',ls='dashed', linewidth=1.5, label = r"Stationary Solution with CA = "+str(contactAngle)+"°")

for idx, wettedAreaFile in enumerate(wettedAreaFileNames):
    pd.set_option('display.float_format', lambda x: '%.1f' % x)
    df = pd.read_csv(wettedAreaFile, index_col = 0)
    plt.xlim([0, 0.04])
    plt.ylim([5.4, 6.2])
    plt.plot(df, label=plotLabels[idx], linewidth=0.85)
    plt.grid(b=True, which='major', linestyle= 'dashed',color='black', alpha=0.5, lw=0.5)
    plt.tick_params(axis='both', direction='out', length=2, width=1.5,  grid_color='black', grid_alpha=0.5)
    #plt.title(r'Mesh convergence study' +'\n'+str(fluid), fontsize=fontSize)
    plt.xlabel(r"time(s)" ,color='black',fontsize=fontSize)
    plt.ylabel(r"Wetted Area($mm^2$)",color='black',fontsize=fontSize)
   # plt.legend(loc='lower center')
    plt.xticks(color='black',size = fontSize)
    plt.yticks(color='black',size = fontSize)
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

### Generate the plot
plt.legend(fontsize=8, loc='lower center', bbox_to_anchor=(0.72, 0), ncol=1)   
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.pdf', bbox_inches='tight')
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.png', bbox_inches='tight')
plt.show()


### Mesh convergence study for water_glycerol with contact angle $\theta_e=110^\circ$

In [ ]:
fluid = 'water_glycerol'
fluidLabel = 'wg'
contactAngle = 110
theta = np.radians(contactAngle)
pattern = fluidLabel+'test'+str(contactAngle)+'_'
dropletRadius = 0.001 #1mm
domainLength = 0.005


nX= [] # static contact angles used for simulations taken from the variation file
stm = [] # surface tension models in all cases,taken from the variation file
var_list = [] # variance number of the parametric study of a specific case

#read the variation-file to get the parameters map that is saved using pyFOAM
var_Map = Funcs.fileMapping(pattern, surfaceTensionModel)
print('variations map: ', var_Map)   
     
for key, value in var_Map.items():
    stm_map = str(value).split(', ')[0][:-1] #surface tension model map
    stm.append(stm_map.split(': ')[1][1:])
    nX_map = str(value).split(', ')[3][:-1] #cells in x-direction map
    #print(theta_map)
    nX.append(int(nX_map.split(': ')[1][0:]))
    var_list.append(str(key))  
print('number of cells in x-direction: ', nX)
print('Variant\'s list: ', var_list)

cellsPerRadius = [int(nx/domainLength*dropletRadius) for nx in nX]
print('cells per radius: ',cellsPerRadius)


#######################################################################################

wettedAreaFileNames = Funcs.fileStructure(dataFolder, 'wettedArea.csv', pattern, var_list )
print (wettedAreaFileNames)
plotLabels = ['cells per radius = ' + str(x) for x in cellsPerRadius]

#Reference solution 
plt.axhline(Funcs.getAnalyticalWettedArea(theta,dropletRadius), color='black',ls='dashed', linewidth=1.5, label = r"Stationary Solution with CA = "+str(contactAngle)+"°")

for idx, wettedAreaFile in enumerate(wettedAreaFileNames):
    pd.set_option('display.float_format', lambda x: '%.1f' % x)
    df = pd.read_csv(wettedAreaFile, index_col = 0)
    #plt.xlim([0, 0.04])
    #plt.ylim([5.4, 6.2])
    plt.plot(df, label=plotLabels[idx], linewidth=0.85)
    plt.grid(b=True, which='major', linestyle= 'dashed',color='black', alpha=0.5, lw=0.5)
    plt.tick_params(axis='both', direction='out', length=2, width=1.5,  grid_color='black', grid_alpha=0.5)
    #plt.title(r'Mesh convergence study' +'\n'+str(fluid), fontsize=fontSize)
    plt.xlabel(r"time(s)" ,color='black',fontsize=fontSize)
    plt.ylabel(r"Wetted Area($mm^2$)",color='black',fontsize=fontSize)
   # plt.legend(loc='lower center')
    plt.xticks(color='black',size = fontSize)
    plt.yticks(color='black',size = fontSize)
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

### Generate the plot
plt.legend(fontsize=8, loc='lower center', bbox_to_anchor=(0.72, 0), ncol=1)   
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.pdf', bbox_inches='tight')
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.png', bbox_inches='tight')
plt.show()


### Mesh convergence study for water with contact angle $\theta_e=70^\circ$

In [ ]:
fluid = 'water'
fluidLabel = 'w'
contactAngle = 70
theta = np.radians(contactAngle)
pattern = fluidLabel+'test'+str(contactAngle)+'_'
dropletRadius = 0.001 #1mm
domainLength = 0.005


nX= [] # static contact angles used for simulations taken from the variation file
stm = [] # surface tension models in all cases,taken from the variation file
var_list = [] # variance number of the parametric study of a specific case

#read the variation-file to get the parameters map that is saved using pyFOAM
var_Map = Funcs.fileMapping(pattern, surfaceTensionModel)
print('variations map: ', var_Map)  
     
for key, value in var_Map.items():
    stm_map = str(value).split(', ')[0][:-1] #surface tension model map
    stm.append(stm_map.split(': ')[1][1:])
    nX_map = str(value).split(', ')[3][:-1] #cells in x-direction map
    #print(theta_map)
    nX.append(int(nX_map.split(': ')[1][0:]))
    var_list.append(str(key))  
print('number of cells in x-direction: ', nX)
print('Variant\'s list: ', var_list)

cellsPerRadius = [int(nx/domainLength*dropletRadius) for nx in nX]
print('cells per radius: ',cellsPerRadius)


#######################################################################################

wettedAreaFileNames = Funcs.fileStructure(dataFolder, 'wettedArea.csv', pattern, var_list )
print (wettedAreaFileNames)
plotLabels = ['cells per radius = ' + str(x) for x in cellsPerRadius]

#Reference solution 
plt.axhline(Funcs.getAnalyticalWettedArea(theta,dropletRadius), color='black',ls='dashed', linewidth=1.5, label = r"Stationary Solution with CA = "+str(contactAngle)+"°")

for idx, wettedAreaFile in enumerate(wettedAreaFileNames):
    pd.set_option('display.float_format', lambda x: '%.1f' % x)
    df = pd.read_csv(wettedAreaFile, index_col = 0)
    #plt.xlim([0, 0.04])
    #plt.ylim([5.4, 6.2])
    plt.plot(df, label=plotLabels[idx], linewidth=0.85)
    plt.grid(b=True, which='major', linestyle= 'dashed',color='black', alpha=0.5, lw=0.5)
    plt.tick_params(axis='both', direction='out', length=2, width=1.5,  grid_color='black', grid_alpha=0.5)
    #plt.title(r'Mesh convergence study' +'\n'+str(fluid), fontsize=fontSize)
    plt.xlabel(r"time(s)" ,color='black',fontsize=fontSize)
    plt.ylabel(r"Wetted Area($mm^2$)",color='black',fontsize=fontSize)
   # plt.legend(loc='lower center')
    plt.xticks(color='black',size = fontSize)
    plt.yticks(color='black',size = fontSize)
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

### Generate the plot
plt.legend(fontsize=8, loc='lower center', bbox_to_anchor=(0.72, 0), ncol=1)   
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.pdf', bbox_inches='tight')
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.png', bbox_inches='tight')
plt.show()


### Mesh convergence study for water with contact angle $\theta_e=110^\circ$

In [ ]:
fluid = 'water'
fluidLabel = 'w'
contactAngle = 110
theta = np.radians(contactAngle)
pattern = fluidLabel+'test'+str(contactAngle)+'_'
dropletRadius = 0.001 #1mm
domainLength = 0.005


nX= [] # static contact angles used for simulations taken from the variation file
stm = [] # surface tension models in all cases,taken from the variation file
var_list = [] # variance number of the parametric study of a specific case

#read the variation-file to get the parameters map that is saved using pyFOAM
var_Map = Funcs.fileMapping(pattern, surfaceTensionModel)
print('variations map: ', var_Map)  
     
for key, value in var_Map.items():
    stm_map = str(value).split(', ')[0][:-1] #surface tension model map
    stm.append(stm_map.split(': ')[1][1:])
    nX_map = str(value).split(', ')[3][:-1] #cells in x-direction map
    #print(theta_map)
    nX.append(int(nX_map.split(': ')[1][0:]))
    var_list.append(str(key))  
print('number of cells in x-direction: ', nX)
print('Variant\'s list: ', var_list)

cellsPerRadius = [int(nx/domainLength*dropletRadius) for nx in nX]
print('cells per radius: ',cellsPerRadius)


#######################################################################################

wettedAreaFileNames = Funcs.fileStructure(dataFolder, 'wettedArea.csv', pattern, var_list )
print (wettedAreaFileNames)
plotLabels = ['cells per radius = ' + str(x) for x in cellsPerRadius]

#Reference solution 
plt.axhline(Funcs.getAnalyticalWettedArea(theta,dropletRadius), color='black',ls='dashed', linewidth=1.5, label = r"Stationary Solution with CA = "+str(contactAngle)+"°")

for idx, wettedAreaFile in enumerate(wettedAreaFileNames):
    pd.set_option('display.float_format', lambda x: '%.1f' % x)
    df = pd.read_csv(wettedAreaFile, index_col = 0)
    #plt.xlim([0, 0.04])
    #plt.ylim([5.4, 6.2])
    plt.plot(df, label=plotLabels[idx], linewidth=0.85)
    plt.grid(b=True, which='major', linestyle= 'dashed',color='black', alpha=0.5, lw=0.5)
    plt.tick_params(axis='both', direction='out', length=2, width=1.5,  grid_color='black', grid_alpha=0.5)
    #plt.title(r'Mesh convergence study' +'\n'+str(fluid), fontsize=fontSize)
    plt.xlabel(r"time(s)" ,color='black',fontsize=fontSize)
    plt.ylabel(r"Wetted Area($mm^2$)",color='black',fontsize=fontSize)
   # plt.legend(loc='lower center')
    plt.xticks(color='black',size = fontSize)
    plt.yticks(color='black',size = fontSize)
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

### Generate the plot
plt.legend(fontsize=8, loc='lower center', bbox_to_anchor=(0.72, 0), ncol=1)   
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.pdf', bbox_inches='tight')
plt.savefig('mesh-convergence-study-'+str(fluid)+'_'+str(contactAngle)+'.png', bbox_inches='tight')
plt.show()
